In [ ]:
#This script opens up the pokemon entry file extracted from Serebii and searches for the relevant information

dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/001_Bulbasaur.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/090_Shellder.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/142_Aerodactyl.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/179_Mareep.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/836_Boltund.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/151_Mew.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/894_Regieleki.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/025_Pikachu.pokedexswsh'


#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/038_Ninetales.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/052_Meowth.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/492_Shaymin.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/555_Darmanitan.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/479_Rotom.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/745_Lycanroc.pokedexswsh'


## ============================================================================================== ##

#Prepares a first loop through the file to identify if the pokemon has Alternate Forms

#Creates a temporary list to store the form names
forms = []

# Reads the data
g = open(dex_entry)

#Starts the loop through the file. Line by line.
for line in g:
    #Finds the pokemon Form lines (only exists in case pokemon has Alternate Forms)
    if line.startswith('			<tr><td class="pkmn"><b>'):

        #Splits line per form using reference string
        all_forms = line.split('loading="lazy" alt="')
        #Finds the total number of forms
        forms_number = len(all_forms) - 1

        #Creates helpful counter to loop through different forms
        i = 1

        #Starts the loop to extract all form names and add to temporary list forms
        while i <= forms_number:
            this_form = all_forms[i].split('"')[0]
            #Adds form name to list
            forms.append(this_form)
            #Moves to next iteration    
            i = i + 1

#If there aren't Alternate Forms
if forms == []:
    forms.append('Only Form')

#print(forms)
    
g.close()


#This is important for the program to know when to stop
c = open(dex_entry)
lines1 = c.readlines()
total_lines = len(lines1)
c.close()

#creates dictionary to store pokemon information
dex_pokemon = {'Species':[], 'Dex':[], 'Forms':{}}

#Sets helpful level_up_moves_final_check variable to 0
#This makes sure the analysis of whether ALL FORMS have assigned Level Up Moves is only conducted once
level_up_moves_final_check = 0 

#Starts second loop to collect all necessary information and store into dictionary

# Reads the data
f = open(dex_entry)

#Starts helpful loop counter
loop_counter = 1

#Starts the loop through the file. Line by line.
while loop_counter <= total_lines:
    line = f.readline()
    loop_counter = loop_counter + 1    
    
    #Finds the pokemon Species and National Dex Number line
    if line.startswith('<title>'):
        this_Species = line.split('>')[1].split(' - #')[0]
        this_Dex = line.split('>')[1].split(' - #')[1].split(' -')[0]
        #print(this_Species)
        #print(this_Dex)
        
        #Handling Exceptions (not taking care of names containing space or : here)
        #Dealing with the nidoran problem
        if 'Nidoran' in this_Species:       
            #For female nidoran
            if '29' in this_Dex:
                this_Species = 'Nidoran F'
            #For male nidoran
            if '32' in this_Dex:
                this_Species = 'Nidoran M'
                
        #Dealing with the pikachu alternate forms problem
        if 'Pikachu' in this_Species:
            #Only considers a single form (ignores the different cap forms)
            forms = []
            forms.append('Only Form')
            #print(forms)            
            
            
        #Adds Species Name to the dicitonary
        dex_pokemon['Species'].append(this_Species)
        print(dex_pokemon['Species'][0])
        #Adds National Dex Number to the dicitonary
        dex_pokemon['Dex'].append(this_Dex)
        print(dex_pokemon['Dex'][0])
        
        
        #Takes care of the different forms
        #Defines/resets helpful counter for forms and 
        forms_counter = 1
        move_counter = {}

        #Sets up the forms into the dictionary
        for current_form in forms:
            #Creates dictionary entries for current form
            dex_pokemon['Forms']['Form'+str(forms_counter)] = {'Name':[], 'Types':[], 'Ability':{}, 'Moves':[]}
            #Adds form name to dictionary
            dex_pokemon['Forms']['Form'+str(forms_counter)]['Name'] = current_form
            #Crestes sub-entries for Ability
            dex_pokemon['Forms']['Form'+str(forms_counter)]['Ability'] = {'Name':[], 'Info':[]}
            #Crestes sub-entries for Moves
            dex_pokemon['Forms']['Form'+str(forms_counter)]['Moves'] = {}
            
            #Creates move_counter dictionary entry for curent form:
            move_counter['Form'+str(forms_counter)] = 1

            #Moves counter to next iteration
            forms_counter = forms_counter + 1        
        
        
    #Finds the pokemon Types lines
    #For single form pokemon or pokemon that all forms share the same types
    #Finding line (this line only exists in files that fit these conditions)
    if line.startswith('			<td class="cen"><a href="') and dex_pokemon['Forms']['Form1']['Types'] == []:
        #Splits line for the first time to identify how many types the pokemon has
        all_types = line.split('/type/')
        #When there is only one type
        if len(all_types) == 2:
            #for type 1
            type_1 = all_types[1].split('.gif"')[0].capitalize()
            #for type 2
            type_2 = ''              
        #When there are two types
        if len(all_types) > 2:
            #for type 1
            type_1 = all_types[1].split('.gif"')[0].capitalize()
            #for type 2
            type_2 = all_types[2].split('.gif"')[0].capitalize()
            
        #Adding type to dictionary inside each form sub-dictionary
        #Defines/resets helpful counter for forms
        forms_counter = 1

        #Loops through the different form sub-dictionaries
        for current_form in forms:
            #Adds Type 1 to current form
            dex_pokemon['Forms']['Form'+str(forms_counter)]['Types'].append(type_1)
            #Adds Type 2 to current form
            dex_pokemon['Forms']['Form'+str(forms_counter)]['Types'].append(type_2)
            #Moves counter to next iteration
            forms_counter = forms_counter + 1  
            

    #For multiple form pokemon
    #Finding line (this line only exists in files that fit these conditions)
    if line.startswith('			<td class="cen"><table') and dex_pokemon['Forms']['Form1']['Types'] == []:

        #The order of the form names from before doesn't always agree with the order of form names for the types
        #so we need a temporary form_types dictionary to help us organize the information before passing to final dicitonary
        temp_forms_types = {}
        
        #print(forms)        

        #In the lines below, we're searaching for the TYPES for EACH FORM and the FORM NAME
            
        #Sometimes the types inforamtion of multiple forms is scattered throughout multiple lines,
        #so we need to use the strategy below to search through multiple lines, combine them into 1 big line
        #and later split the big line into a line for each form
        
        #Creates/resets helpful variables big_line, line_count, line_end
        big_line = ""
        line_count = 0
        line_end = 5

        #Looks for first form's name or '"50%">Normal', which they normally use to refer to first form
        if current_form in line or '"50%">Normal' in line:
            #print('here!!!!!')
            #print("line 0 (searchable line):")
            #print(line)
            #Adds line to big_line:
            big_line = big_line + line
            #Moves to next iteration
            line_count = line_count + 1

            #Gets the following x lines (specified by the helpful variable line_end) and combines them into 1 big line
            #(sometimes the inforamtion of multiple forms is scattered throughout multiple lines)
            while line_count <= line_end:
                #Moves to next line
                line = f.readline()
                #print("line "+str(line_count)+":")
                #print(line)
                #Adds line to big_line:
                big_line = big_line + line
                #Moves to next iteration
                line_count = line_count + 1
            
            #print(big_line)
            
            #Splits big_line using </td><td to extract information for FORM NAME
            line_form_name = big_line.split('</td><td')
            #Deletes last element since it has no FORM NAME information
            del line_form_name[-1]
            #Creates/resets helpful counter for form names
            counter_form_names = 1            
            #Goes through each part to extract form name
            for part in line_form_name:
                #Creates entry on temporary form_types dictionary for current form
                temp_forms_types['Form'+str(counter_form_names)] = {'Name':[],'Types':[]}
                #Splits again using ="50%"> and adds the form name from the last part to the temporary dictionary
                temp_forms_types['Form'+str(counter_form_names)]['Name'].append(part.split('="50%">')[-1])
                #Moves counter tok next iteration
                counter_form_names = counter_form_names + 1
                
            
            #Splits big_line according to the number of forms using </td><td , which appears after every form name
            #The TYPES normally follow the form name, so part 0 doesn't have any types information
            
            #Creates/resets helpful counter form_number
            form_number = 0
            
            while form_number < len(forms):                
                
                #Splits big_line using </td><td to extract information for current form's TYPES
                #which is in part i+1, since part 0 doesn't have any types information
                line = big_line.split('</td><td')[form_number+1]
                #print(line)
                
                #Here we further split big_line_split to look for the types information:
                #Splits line for the first time to identify how many types the pokemon has
                all_types = line.split('/type/')
                #When there is only one type
                if len(all_types) == 2:
                    #for type 1
                    type_1 = all_types[1].split('.gif"')[0].capitalize()
                    #for type 2
                    type_2 = ''              
                #When there are two types
                if len(all_types) > 2:
                    #for type 1
                    type_1 = all_types[1].split('.gif"')[0].capitalize()
                    #for type 2
                    type_2 = all_types[2].split('.gif"')[0].capitalize()
                    
                #Adds current form's types to temporary dictionary
                temp_forms_types['Form'+str(form_number+1)]['Types'].append(type_1)
                temp_forms_types['Form'+str(form_number+1)]['Types'].append(type_2)

                #print(forms[form_number])
                #print(type_1)
                #print(type_2)                    
                
                #Moves to next iteration
                form_number = form_number + 1
                
                #print(big_line_split)                
            #print(temp_forms_types)

        
        #Approach 1: When FORM NAMES are written the same, but the order is different
        #Compares temporary dictionary to actual dictionary to finally add FORM TYPES
        #Starts to loop through temporary dictionary
        for item in temp_forms_types:
            #print(' ')
            #print(temp_forms_types[item]['Name'][0])
            #Starts to loop through actual dictionary  dex_pokemon['Forms'] to check for a form name match
            for item_final in dex_pokemon['Forms']:
                #print(dex_pokemon['Forms'][item_final]['Name'])
                #Looking for the TYPE NAME match
                if temp_forms_types[item]['Name'][0] == dex_pokemon['Forms'][item_final]['Name']:
                    #Adds types to actual dictionary dex_pokemon['Forms']
                    dex_pokemon['Forms'][item_final]['Types'].append(temp_forms_types[item]['Types'][0])
                    dex_pokemon['Forms'][item_final]['Types'].append(temp_forms_types[item]['Types'][1])
                    
                    
        #Approach 2: When FORM NAMES are NOT written the same, we have to assume the order is the same
        #Compares actual dictionary to temporary dictionary to finally add FORM TYPES
        #Starts to loop through actual dictionary dex_pokemon['Forms']
        for item_final in dex_pokemon['Forms']:
            #Checks if TYPES is empty so we need to switch to Approach 2 (equal order)
            if dex_pokemon['Forms'][item_final]['Types'] == []:
                #Adds types to actual dictionary dex_pokemon['Forms']
                dex_pokemon['Forms'][item_final]['Types'].append(temp_forms_types[item_final]['Types'][0])
                dex_pokemon['Forms'][item_final]['Types'].append(temp_forms_types[item_final]['Types'][1])
            
               

    #Starts looking for abilities for both SINGLE FORM and MULTIPLE FORM pokemon    
    
    #For SINGLE FORM Pokemon:
    if len(dex_pokemon['Forms']) == 1:
        #For Regular Abilities
        if line.startswith('<a href="/abilitydex/'):
            this_Ability = line.split('><b>')[1].split('</b>')[0]
            this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
            #Adds Ability Name and Info to the dicitonary
            dex_pokemon['Forms']['Form1']['Ability']['Name'].append(this_Ability)
            dex_pokemon['Forms']['Form1']['Ability']['Info'].append(this_Ability_Info)
            #print(dex_pokemon['Forms']['Form1']['Ability'])
        #For Hidden Abilities
        if line.startswith('<b>Hidden Ability</b><!-- <i>(Available)'):        
            #Checks if this is a Hidden Ability
            this_Ability = line.split('><b>')[1].split('</b>')[0] + ' (Hidden Ability)'
            this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
            #Adds Ability Name and Info to the dicitonary
            dex_pokemon['Forms']['Form1']['Ability']['Name'].append(this_Ability)
            dex_pokemon['Forms']['Form1']['Ability']['Info'].append(this_Ability_Info)
            #print(dex_pokemon['Forms']['Form1']['Ability'])

            
    #For MULTIPLE FORM Pokemon:
    #Creates/resets helpful form counter
    counter_forms_abilities = 1
    
    if len(dex_pokemon['Forms']) > 1:
        #Finding first ability line:
        if line.startswith('<a href="/abilitydex/'):
            #Adds first ability to first form
            this_Ability = line.split('><b>')[1].split('</b>')[0]
            this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
            #Adds Ability Name and Info to first form
            dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'].append(this_Ability)
            dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'].append(this_Ability_Info)
            #print(dex_pokemon['Forms']['Form1']['Ability'])
            
            #Starts a sub-loop to go through ability lines
            while line.startswith('		</tr>') == False:
                #Moves to next line
                line = f.readline()
                #print(line)

                #Checks if next line is a regular ability line
                if line.startswith('<a href="/abilitydex/'):
                    #Adds ability to current form
                    this_Ability = line.split('><b>')[1].split('</b>')[0]
                    this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #Adds Ability Name and Info to current form
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'].append(this_Ability)
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'].append(this_Ability_Info)
                    
                #Checks if next line is a hidden ability line
                if line.startswith('<b>Hidden'):        
                    #Checks if this is a Hidden Ability
                    this_Ability = line.split('><b>')[1].split('</b>')[0] + ' (Hidden Ability)'
                    this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #Adds Ability Name and Info to current form
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'].append(this_Ability)
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'].append(this_Ability_Info)
                    
                #Checks if next line is a NEW FORM line 
                #since both NEW FORM lines and hidden ability lines start very similarly, we have to be more selective
                if line.startswith('<b>') == True and line.startswith('<b>Hidden') == False:
                    #Extracts form name
                    form_name = line.split(' Abilit')[0].strip('<b>')
                    #form_name = line.split(' Abilities</b>')[0].strip('<b>') #Had to change line due to plural/singular discrepancy
                    #print(form_name)
                    
                    #Moves to the form with the MATCHING FORM NAME
                    #Sometimes more than one form share the same abilities and the abilities are only displayed once
                    #So we assume that the abilites are the same for the previous forms until we find a MATCH
                    #Moves to NEXT FORM
                    counter_forms_abilities = counter_forms_abilities + 1  
                    #If there is no MATCH, we assume that the abilites are the same for the previous forms until we find a MATCH
                    while form_name not in dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Name']:
                        #Copies Ability Name and Info to current form from previous form
                        dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'] = dex_pokemon['Forms']['Form'+str(counter_forms_abilities-1)]['Ability']['Name']
                        dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'] = dex_pokemon['Forms']['Form'+str(counter_forms_abilities-1)]['Ability']['Info']
                        
                        #Moves to NEXT FORM
                        counter_forms_abilities = counter_forms_abilities + 1 
        
                    #If there is a MATCH
                    #Adds ability to current form
                    this_Ability = line.split('><b>')[1].split('</b>')[0]
                    this_Ability_Info = line.split('></a>: ')[1].split(' <')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #Adds Ability Name and Info to current form
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'].append(this_Ability)
                    dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'].append(this_Ability_Info)
                    
                    
                    
    ############### For SINGLE FORM POKEMON - Moves
    if len(dex_pokemon['Forms']) == 1:

        #Finding Level Up Moves Title Line
        if 'Level Up</h3>' in line and '"legendsattacks"' not in line:
            #Creates/Updates current_method and number_info variable to Level Up
            current_method = 'Level up'
            number_info = 'Level '

            #Moves to next line
            line = f.readline()

            #Starts a sub-loop through the lines until the end of the Level up block
            while line.startswith('	<td class="foox"') == False and '<td rowspan="2" class="fooinfo"><a' not in line and line.startswith('				<tr><td class="fooinfo"') == False:


                #Extracts move information using a loop of 9 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('>')[1].split('<')[0]
                #When there is no level 
                if this_number == '&#8212;':
                    this_number = '-'
                #print(number_info+this_number)
                #When learns move upon evolving
                if this_number == 'Evolve' or this_number == '-':
                    #Adds to current form
                    dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Number'].append(this_number)
                else:
                    #Adds to current form
                    dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Number'].append(number_info+this_number)

                #2)For the Move Name:
                #Moves to next line
                #print(line)
                line = f.readline()
                #Extracts Name
                #print(line)
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)

                #Moves to next line only if
                if '.</td></tr><tr>' in line:
                    line = f.readline()
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1
                

        #Finding Sword & Shield Technical Machine Attacks Title Line
        if 'Sword & Shield Technical Machine Attacks' in line or '>Technical Machine Attacks<' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Sword & Shield TMs'
            #number_info = 'TM' 

            #Starts a sub-loop through the lines until the end of the Sword & Shield TMs block
            while 'Technical Record Attacks' not in line and '</a><h3>Egg Moves</h3>' not in line:

                #Extracts move information using a loop of 9 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Number'].append(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)
                
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1


        #Finding Technical Record Attacks Title Line
        if 'Technical Record Attacks' in line:
            #Creates/Updates current_method and number_info variable to TRs
            current_method = 'TRs'
            number_info = 'TR' 

            #Starts a sub-loop through the lines until the end of the TRs block
            while 'BDSP Technical Machine Attacks' not in line and '</a><h3>Egg Moves</h3>' not in line and 'Move Tutor Attacks<' not in line:

                #Extracts move information using a loop of 9 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Number'].append(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)
                
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1

                
        #Finding BDSP Technical Machine Attacks Title Line
        if 'BDSP Technical Machine Attacks' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'BDSP TMs'
            number_info = 'TM' 

            #Starts a sub-loop through the lines until the end of the TRs block
            while '</a><h3>Egg Moves</h3>' not in line and 'Move Tutor Attacks<' not in line and '"transfer"' not in line:

                #Extracts move information using a loop of 9 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Number'].append(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)    
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)
                
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1

        #Finding Egg Moves Title Line
        if '</a><h3>Egg Moves</h3>' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Egg Moves' 

            #Starts a sub-loop through the lines until the end of the TRs block
            while 'Move Tutor Attacks<' not in line and 'Usable Max Moves' not in line and 'name="transfer"' not in line:

                #Extracts move information using a loop of 8 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the Move Name:
                #Extracts Name
                #For this first Egg Move
                if 'egg.shtml">' in line:
                    this_name = line.split('egg.shtml">')[1].split('.shtml">')[1].split('<')[0]
                #For following Egg Moves
                if 'egg.shtml">' not in line:
      ####              ###print(line)
                    this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #2)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #3)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #4)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)      

                #5)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #6)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #7)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #8)For the Info:
                #Moves to next line (skipping the 'Details' line)
                line = f.readline()
                #Moves to next line (skipping the blank line)
                line = f.readline()
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)   
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)
                
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1


        #Finding Move Tutor Attacks Title Line
        if 'Move Tutor Attacks<' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Move Tutor'

            #print(line)

            #Splits the line right in front of move name to separate line by moves
            split_line = line.split('.shtml">')
            #Deletes first element from the list
            split_line.pop(0)

            #Creates a loop to use all elements of split_line, except for element 0
            for element in split_line:

                #Extracts move information using line splitting
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Method'].append(current_method)

                #1)For the Move Name:
                #Extracts Name
                this_name = element.split('</a></')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Name'].append(this_name)

                #2)For the Move Type:
                #Extracts Type
                this_type = element.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Type'].append(this_type)

                #3)For the Move Mode:
                #Extracts Mode
                this_mode = element.split('/type/')[2].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Mode'].append(this_mode)

                #4)For the Power:
                #Extracts Power
                this_power = element.split('"fooinfo">')[3].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Power'].append(this_power)

                #5)For the Accuracy:
                #Extracts Accuracy
                this_accuracy = element.split('"fooinfo">')[4].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Accuracy'].append(this_accuracy)

                #6)For the PP:
                #Extracts PP
                this_pp = element.split('"fooinfo">')[5].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['PP'].append(this_pp)

                #7)For the Effect %:
                #Extracts Effect %
                this_effect = element.split('"fooinfo">')[6].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Effect'].append(this_effect)

                #8)For the Info:
                #Extracts Info
                this_info = element.split('"fooinfo">')[7].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)   
                #Adds to current form
                dex_pokemon['Forms']['Form1']['Moves']['Move'+str(move_counter['Form1'])]['Info'].append(this_info)
                
                #Moves move_counter to next iteration
                move_counter['Form1'] = move_counter['Form1'] + 1

    
    ############### For MULTI FORM POKEMON - Moves
    if len(dex_pokemon['Forms']) > 1:

        #Finding Level Up Moves Title Lines (for all the forms)
        while 'Level Up</h3>' in line and '"legendsattacks"' not in line:
            #Creates/Updates current_method and number_info variable to Level Up
            current_method = 'Level up'
            number_info = 'Level '
            
            #Identifying the form
            current_form = line.split('level"></a>')[1].split(' Level')[0].strip(' Form')
            #Fixes the Alolan/Alola Form discrepancy
            if current_form == 'Alolan':
                current_form = 'Alola' #the shorter form will always work when using the in statement
            #print(current_form)
            
            #Finding the corresponding form within the dictionary
            #Creating/resetting helpful loop counter:
            form_counter = 1
            
            #When dealing with the Standard form  (assumes standard form to be Form1)
            if current_form == 'Standard':
                form_counter = 1
                #print('yes')
                #print(form_counter)
            
            #When not dealing with standard form
            if current_form != 'Standard':
                #Starting loop
                while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                    #When not a match, moves counter to next iteration
                    form_counter = form_counter + 1
                    #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                #When a match, does nothing (this is only used for debugging)
                #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                    #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                    #print('yes')
                    #print(form_counter)
                        
            #Moves to next line
            line = f.readline()

            #Starts a sub-loop through the lines until the end of the Level up block
            while line.startswith('	<td class="foox"') == False and '<td rowspan="2" class="fooinfo"><a' not in line and line.startswith('				<tr><td class="fooinfo"') == False:

                #Extracts move information using a loop of 9 lines for each move
                
                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}
                
                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('>')[1].split('<')[0]
                #When there is no level 
                if this_number == '&#8212;':
                    this_number = '-'
                #print(number_info+this_number)
                #When learns move upon evolving
                if this_number == 'Evolve' or this_number == '-':
                    #Adds to current form
                    dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(this_number)
                else:
                    #Adds to current form
                    dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(number_info+this_number)

                #2)For the Move Name:
                #Moves to next line
                #print(line)
                line = f.readline()
                #Extracts Name
                #print(line)
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)
                
                #Moves to next line only if
                if '.</td></tr><tr>' in line:
                    line = f.readline()
                #Moves move_counter to next iteration
                move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                
        ### Dealing with the Level Up - XXX Form Title Line Discrepancy
        #Finding Level Up Moves Title Lines (for all forms with the different title)
        while 'Level Up - ' in line and '"legendsattacks"' not in line:        

            #Identifying the form
            current_form = line.split('Level Up - ')[1].split('<')[0].strip(' Forme').strip(' Form')
            print(current_form)

            #Finding the corresponding form within the dictionary
            #Creating/resetting helpful loop counter:
            form_counter = 1

            #Starting loop
            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                #When not a match, moves counter to next iteration
                form_counter = form_counter + 1
                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
            #When a match, does nothing (this is only used for debugging)
            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                #print('yes')
                #print(form_counter)

            #print(line)
            #Moves to the next line
            line = f.readline()
            #Starts a sub-loop through the lines until the end of the Level up block
            while line.startswith('	<td class="foox"') == False and '<td rowspan="2" class="fooinfo"><a' not in line and line.startswith('				<tr><td class="fooinfo"') == False:

                #Extracts move information using a loop of 9 lines for each move

                #Sets up sub dictionary entry for a new move
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}

                #0)Adds Method to this set of moves:
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)

                #1)For the number:
                #Extracts number
                this_number = line.split('>')[1].split('<')[0]
                #When there is no level 
                if this_number == '&#8212;':
                    this_number = '-'
                #print(number_info+this_number)
                if this_number == 'Evolve' or this_number == '-':
                    #Adds to current form
                    dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(this_number)                            
                else:
                    #Adds to current form
                    dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(number_info+this_number)

                #2)For the Move Name:
                #Moves to next line
                #print(line)
                line = f.readline()
                #Extracts Name
                #print(line)
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                #Adds to current form
                dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)

                #Moves to next line only if
                if '.</td></tr><tr>' in line:
                    line = f.readline()
                #Moves move_counter to next iteration
                move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1

                
#########Final check to make sure ALL FORMS have assigned Level Up Moves  (May not be the fastest way to do this...)
        #This is strategically positioned so only Level up Moves were loaded at this point
        #This is very important for when MULTIPLE FORMS share the same Level Up Moves and the Level Up Moves are only stated once (e.g. Darmanitan and Rotom)
        #This analysis is conducted only once and AFTER THE LEVEL UP MOVES WERE ADDED
        if level_up_moves_final_check == 0 and len(dex_pokemon['Forms']['Form1']['Moves']) > 1:
            #Makes sure this analysis is only conducted only once by setting variable to 1
            level_up_moves_final_check = 1  

            #Creates/resets helpful form counter
            counter_forms_level_up_moves = 1
            #Starts to loop through dictionary
            while counter_forms_level_up_moves <= len(dex_pokemon['Forms']):

                #Sets/Resets useful variable to 0 (meaning there are NO Level up Moves for current form)
                level_up_moves = 0
                #Loops through moves within form
                for moves in dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves)]['Moves']:
                    #If current form has at least one Level up Move
                    if dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves)]['Moves'][moves]['Method'][0] == 'Level up':
                        level_up_moves = 1

                #If there were No Level up moves
                if level_up_moves == 0:
                    #print('No')  
                    #Copies information from the previous FORM (Assumes Level Up Moves to be the same between the two FORMS)
                    #When working with Darmamitan, we want to forever link Zen mode and Normal mode's moves for each regional variant so we don't have to worry about copying TMs/TRs/Move tutor/Egg moves
                    if dex_pokemon['Species'][0] == 'Darmanitan':  
                        dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves)]['Moves'] = dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves-1)]['Moves']  
                        #This ^ forever links the two dictionaries = when one changes, the other automatically changes
                    #When not working with Darmanitan, we don't want to forever link forms' moves because they may vary later on (e.g. move tutor, special moves)
                    if dex_pokemon['Species'][0] != 'Darmanitan':
                        dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves)]['Moves'] = dict(dex_pokemon['Forms']['Form'+str(counter_forms_level_up_moves-1)]['Moves'])  
                        #This ^ copies the info from one dict to the other without linking them forever
                    #Updates move_counter to appropriate number, so more moves can be added without replacing previous ones
                    move_counter['Form'+str(counter_forms_level_up_moves)] = move_counter['Form'+str(counter_forms_level_up_moves-1)]
                    #This ^ works here because we're refering directly to the value, in this case an int
                #Moves to next iteration
                counter_forms_level_up_moves = counter_forms_level_up_moves + 1        
                
        
        #Finding Sword & Shield Technical Machine Attacks Title Line
        if 'Sword & Shield Technical Machine Attacks' in line or '>Technical Machine Attacks<' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Sword & Shield TMs'
            #number_info = 'TM' 
            
            #Checks if TMs are separated by FORMS (e.g. differennt forms learn different moves)
            if '>Form<' in line:
                #Sets/resets individual_form_TM_TR to 1:
                individual_form_TM_TR = 1
            else:
                #Sets/resets individual_form_TM_TR to 0:
                individual_form_TM_TR = 0

            #Starts a sub-loop through the lines until the end of the Sword & Shield TMs block
            while 'Technical Record Attacks' not in line and '</a><h3>Egg Moves</h3>' not in line:          

                #Extracts move information using a loop of 10 lines for each move

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                
                #8.5)For individual form information
                if individual_form_TM_TR == 1:
                    #Moves to next line
                    line = f.readline()
                    #Extracts FORM info
                    #Makes first split before the names of the forms
                    this_form_info = line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0].strip(' Form')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)
                    
                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                
                #Starts a loop to add move information to the dictitionary for the corresponding FORMS
                #For individual form information
                if individual_form_TM_TR == 1:
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)                                    
                                    
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                        
                #When there is no individual form information, we assume all forms can learn these moves
                if individual_form_TM_TR == 0:
                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1
                        
                        
        #Finding Technical Record Attacks Title Line
        if 'Technical Record Attacks' in line:
            #Creates/Updates current_method and number_info variable to TRs
            current_method = 'TRs'
            number_info = 'TR' 
            
            #Checks if TRs are separated by FORMS (e.g. differennt forms learn different moves)
            if '>Form<' in line:
                #Sets/resets individual_form_TM_TR to 1:
                individual_form_TM_TR = 1
            else:
                #Sets/resets individual_form_TM_TR to 0:
                individual_form_TM_TR = 0

            #Starts a sub-loop through the lines until the end of the TRs block
            while 'BDSP Technical Machine Attacks' not in line and '</a><h3>Egg Moves</h3>' not in line and 'Move Tutor Attacks<' not in line:

                #Extracts move information using a loop of 10 lines for each move

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                
                #8.5)For individual form information
                if individual_form_TM_TR == 1:
                    #Moves to next line
                    line = f.readline()
                    #Extracts FORM info
                    #Makes first split before the names of the forms
                    this_form_info = line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0].strip(' Form')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)

                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)
                
                
                #Starts a loop to add move information to the dictitionary for the corresponding FORMS
                #For individual form information
                if individual_form_TM_TR == 1:
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)                                    
                                    
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                        
                #When there is no individual form information, we assume all forms can learn these moves
                if individual_form_TM_TR == 0:
                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1

                        
        #Finding BDSP Technical Machine Attacks Title Line - NO REGIONAL VARIANTS APPEAR IN BDSP
        if 'BDSP Technical Machine Attacks' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'BDSP TMs'
            number_info = 'TM' 
            
            #Checks if TMs are separated by FORMS (e.g. differennt forms learn different moves)
            if '>Form<' in line:
                #Sets/resets individual_form_TM_TR to 1:
                individual_form_TM_TR = 1
            else:
                #Sets/resets individual_form_TM_TR to 0:
                individual_form_TM_TR = 0

            #Starts a sub-loop through the lines until the end of the TRs block
            while '</a><h3>Egg Moves</h3>' not in line and 'Move Tutor Attacks<' not in line and '"transfer"' not in line:

                #Extracts move information using a loop of 9 lines for each move

                #1)For the number:
                #Extracts number
                this_number = line.split('.shtml">')[1].split('<')[0]
                #print(this_number)

                #2)For the Move Name:
                #Moves to next line
                line = f.readline()
                #Extracts Name
                this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)

                #3)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)

                #4)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)

                #5)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)

                #6)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)

                #7)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)

                #8)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)

                #8.5)For individual form information
                if individual_form_TM_TR == 1:
                    #Moves to next line
                    line = f.readline()
                    #Extracts FORM info
                    #Makes first split before the names of the forms
                    this_form_info = line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0].strip(' Form')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)
                
                #9)For the Info:
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)    
                                
                #Starts a loop to add move information to the dictitionary for the corresponding FORMS
                #For individual form information
                if individual_form_TM_TR == 1:
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)                                    
                                    
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                        
                #When there is no individual form information, we assume all forms can learn these moves
                if individual_form_TM_TR == 0:
                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':[], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Number to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Number'].append(this_number)                
                        #2)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #3)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #4)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #5)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #6)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #7)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #8)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #9)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1
                        

        #Finding Egg Moves Title Line
        if '</a><h3>Egg Moves</h3>' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Egg Moves' 

            #Starts a sub-loop through the lines until the end of the TRs block
            while 'Move Tutor Attacks<' not in line and 'Usable Max Moves' not in line and 'name="transfer"' not in line:

                #Sets/resets individual_form_egg_move to 0:
                #This variable is specially useful when different forms learn different egg moves
                individual_form_egg_move = 0                
                
                #Extracts move information using a loop of 8 lines for each move

                #1)For the Move Name:
                #Extracts Name
                #For this first Egg Move
                if 'egg.shtml">' in line:
                    this_name = line.split('egg.shtml">')[1].split('.shtml">')[1].split('<')[0]
                #For following Egg Moves
                if 'egg.shtml">' not in line:
      ####              ###print(line)
                    this_name = line.split('.shtml">')[1].split('<')[0]
                #print(this_name)

                #2)For the Move Type:
                #Moves to next line
                line = f.readline()
                #Extracts Type
                this_type = line.split('/type/')[1].split('.gif')[0].capitalize()
                #print(this_type)

                #3)For the Move Mode:
                #Moves to next line
                line = f.readline()
                #Extracts Mode
                this_mode = line.split('/type/')[1].split('.png')[0].capitalize()
                #When it's a Status Move 
                if this_mode == 'other' or this_mode == 'Other':
                    this_mode = 'Status'
                #print(this_mode)

                #4)For the Power:
                #Moves to next line
                line = f.readline()
                #Extracts Power
                this_power = line.split('>')[1].split('<')[0]
                #print(this_power)      

                #5)For the Accuracy:
                #Moves to next line
                line = f.readline()
                #Extracts Accuracy
                this_accuracy = line.split('>')[1].split('<')[0]
                #print(this_accuracy)

                #6)For the PP:
                #Moves to next line
                line = f.readline()
                #Extracts PP
                this_pp = line.split('>')[1].split('<')[0]
                #print(this_pp)

                #7)For the Effect %:
                #Moves to next line
                line = f.readline()
                #Extracts Effect %
                this_effect = line.split('>')[1].split('<')[0]
                #print(this_effect)
                
                #7.5)For individual FORM information
                #Moves to next line (the 'Details' line)
                line = f.readline()
                #Checking if there is a .png for a FORM here, if so FORMS learn individual egg moves
                if '.png"' in line:
                    #Sets individual_form_egg_move to 1
                    individual_form_egg_move = 1
                    #Extracts FORM info for current egg move
                    #Makes first split before the names of the forms
                    this_form_info = line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0].strip(' Form')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)         
                    
                #8)For the Info:
                #Moves to next line (skipping the blank line)
                line = f.readline()
                #Moves to next line
                line = f.readline()
                #Extracts Info
                this_info = line.split('colspan="6">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                #print(this_info)   
                
                #Starts a loop to add move information to the dictitionary for the corresponding FORMS
                #For individual form information
                if individual_form_egg_move == 1:
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)                                    
                                    
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                        
                #When there is no individual form information, we assume ALL FORMS can learn these moves
                if individual_form_egg_move == 0:
                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1
                        
                        
        #Finding Move Tutor Attacks Title Line
        if 'Move Tutor Attacks<' in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Move Tutor'
            
            #Checks if moves are separated by FORMS (e.g. differennt forms learn different moves)
            if 'attheader">Form' in line:
                #Sets/resets individual_form_move_tutor to 1:
                individual_form_move_tutor = 1
            else:
                #Sets/resets individual_form_move_tutor to 0:
                individual_form_move_tutor = 0
                
            #When moves are separated by FORMS, the information no longer comes in a single line
            if individual_form_move_tutor == 1:
                
                #Starts a sub-loop through the lines until the end of the move tutor block
                while 'Usable Max Moves' not in line and 'name="transfer"' not in line and '>Special Moves<' not in line:

                    #Extracts move information                    
                    
                    #Splits the line right in front of move name
                    element = line.split('.shtml">')[1]

                    #1)For the Move Name:
                    #Extracts Name
                    this_name = element.split('</a></')[0]
                    #print(this_name)

                    #2)For the Move Type:
                    #Extracts Type
                    this_type = element.split('/type/')[1].split('.gif')[0].capitalize()
                    #print(this_type)

                    #3)For the Move Mode:
                    #Extracts Mode
                    this_mode = element.split('/type/')[2].split('.png')[0].capitalize()
                    #When it's a Status Move 
                    if this_mode == 'other' or this_mode == 'Other':
                        this_mode = 'Status'
                    #print(this_mode)

                    #4)For the Power:
                    #Extracts Power
                    this_power = element.split('"fooinfo">')[3].split('<')[0]
                    #print(this_power)

                    #5)For the Accuracy:
                    #Extracts Accuracy
                    this_accuracy = element.split('"fooinfo">')[4].split('<')[0]
                    #print(this_accuracy)

                    #6)For the PP:
                    #Extracts PP
                    this_pp = element.split('"fooinfo">')[5].split('<')[0]
                    #print(this_pp)

                    #7)For the Effect %:
                    #Extracts Effect %
                    this_effect = element.split('"fooinfo">')[6].split('<')[0]
                    #print(this_effect)

                    #7.5)For the FORM info
                    #Moves to next line
                    line = f.readline()
                    #Gets the FORM info part of the line
                    form_info_line = line.split('></table></')[0]
                    #Extracts FORM info
                    #Makes first split before the names of the forms
                    this_form_info = form_info_line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0].strip(' Form')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Fixes the Standard replaced by Pokemon Name discrepancy
                        if form == dex_pokemon['Species'][0]:
                            form = 'Standard' #replacing the pokemon name with Standard makes the code more universal                        
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)                    

                    #8)For the Info:
                    #When the pokemon learns from 2 different move tutors, the line breaks into 2 after forms info
                    if '"fooinfo">' not in line:
                        #Moves to next line
                        line = f.readline()                        
                    #Extracts Info
                    this_info = line.split('"fooinfo">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #print(this_info)   
                    
                    #Starts a loop to add move information to the dictitionary for the corresponding FORMS for individual form information
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)                                    

                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                

                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                

            #When moves are not separated by FORMS, we assume ALL FORMS can leaarn the moves
            if individual_form_move_tutor == 0:

                #Splits the line, first close to the Move Tutor title to avoid information from the following session (e.g. Max Moves, Special Moves for Rotom)
                #second, right in front of move name to separate line by moves
                split_line = line.split('">Attack Name</')[1].split('.shtml">')
                #Deletes first element from the list
                split_line.pop(0)

                #Creates a loop to use all elements of split_line, except for element 0
                for element in split_line:

                    #Extracts move information using line splitting

                    #1)For the Move Name:
                    #Extracts Name
                    this_name = element.split('</a></')[0]
                    #print(this_name)

                    #2)For the Move Type:
                    #Extracts Type
                    this_type = element.split('/type/')[1].split('.gif')[0].capitalize()
                    #print(this_type)

                    #3)For the Move Mode:
                    #Extracts Mode
                    this_mode = element.split('/type/')[2].split('.png')[0].capitalize()
                    #When it's a Status Move 
                    if this_mode == 'other' or this_mode == 'Other':
                        this_mode = 'Status'
                    #print(this_mode)

                    #4)For the Power:
                    #Extracts Power
                    this_power = element.split('"fooinfo">')[3].split('<')[0]
                    #print(this_power)

                    #5)For the Accuracy:
                    #Extracts Accuracy
                    this_accuracy = element.split('"fooinfo">')[4].split('<')[0]
                    #print(this_accuracy)

                    #6)For the PP:
                    #Extracts PP
                    this_pp = element.split('"fooinfo">')[5].split('<')[0]
                    #print(this_pp)

                    #7)For the Effect %:
                    #Extracts Effect %
                    this_effect = element.split('"fooinfo">')[6].split('<')[0]
                    #print(this_effect)

                    #8)For the Info:
                    #Extracts Info
                    this_info = element.split('"fooinfo">')[7].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #print(this_info)   

                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1
     
    ##################### STOPPED HERE!!! #Special Moves for Rotom is done!!!
    
        #Finding Special Moves Title Line (So far, I only saw this for Rotom and its forms)
        if '="fooevo">Special Moves<' in line and '"legendsattacks"' not in line and '>Move Shop Attacks<' not in line:
            #Creates/Updates current_method and number_info variable to TMs
            current_method = 'Special Moves'
            
            #Checks if moves are separated by FORMS (e.g. differennt forms learn different moves)
            if '90">Form<' in line:
                #Sets/resets individual_form_special_moves to 1:
                individual_form_special_moves = 1
            else:
                #Sets/resets individual_form_special_moves to 0:
                individual_form_special_moves = 0
                
            #When moves are separated by FORMS, the information no longer comes ina  single line
            if individual_form_special_moves == 1:
                
                #Splits the line, first right at the Special Moves title to avoid information from the session before 
                line = line.split('="fooevo">Special Moves<')[1]
                
                #Starts a sub-loop through the lines until the end of the move tutor block
                while 'Usable Max Moves' not in line and 'name="transfer"' not in line:

                    #Extracts move information                  
                       
                    #Splits the line, right in front of move name to start from the move name
                    element = line.split('.shtml">')[1]
                    #print(element)

                    #1)For the Move Name:
                    #Extracts Name
                    this_name = element.split('</a></')[0]
                    #print(this_name)

                    #2)For the Move Type:
                    #Extracts Type
                    this_type = element.split('/type/')[1].split('.gif')[0].capitalize()
                    #print(this_type)

                    #3)For the Move Mode:
                    #Extracts Mode
                    this_mode = element.split('/type/')[2].split('.png')[0].capitalize()
                    #When it's a Status Move 
                    if this_mode == 'other' or this_mode == 'Other':
                        this_mode = 'Status'
                    #print(this_mode)

                    #4)For the Power:
                    #Extracts Power
                    this_power = element.split('"fooinfo">')[3].split('<')[0]
                    #print(this_power)

                    #5)For the Accuracy:
                    #Extracts Accuracy
                    this_accuracy = element.split('"fooinfo">')[4].split('<')[0]
                    #print(this_accuracy)

                    #6)For the PP:
                    #Extracts PP
                    this_pp = element.split('"fooinfo">')[5].split('<')[0]
                    #print(this_pp)

                    #7)For the Effect %:
                    #Extracts Effect %
                    this_effect = element.split('"fooinfo">')[6].split('<')[0]
                    #print(this_effect)

                    #7.5)For the FORM info
                    #Moves to next line
                    line = f.readline()
                    #Gets the FORM info part of the line
                    form_info_line = line.split('.</td><td')[0]
                    #Extracts FORM info
                    #Makes first split before the names of the forms
                    this_form_info = form_info_line.split('alt="')
                    #Removes initial text
                    this_form_info.pop(0)
                    #Creates/resets this_form_list
                    this_form_list = []
                    #Makes the second split to isolate form names and adds them to this_form_list
                    for form in this_form_info:
                        #Prepares form
                        form = form.split('"')[0]
                        #If the word Form is in the form name text (e.g. Rotom is not using the word Form after the form names)
                        if 'Form' in form:
                            form = form.strip(' Form')
                        #Removes the word Rotom from Rotom Form names to facilitate for the program
                        if 'Rotom' in form:
                            form = form.replace(' Rotom','')
                        #Fixes the Alolan/Alola Form discrepancy
                        if form == 'Alolan':
                            form = 'Alola' #the shorter form will always work when using the in statement
                        #Fixes the Standard replaced by Pokemon Name discrepancy
                        if form == dex_pokemon['Species'][0]:
                            form = 'Standard' #replacing the pokemon name with Standard makes the code more universal                        
                        #Adds form to list
                        this_form_list.append(form)
                    #print(this_form_list)                    

                    #8)For the Info:
                    #When the pokemon learns from 2 different move tutors, the line breaks into 2 after forms info
                    if '"fooinfo">' not in line:
                        #Moves to next line
                        line = f.readline()                        
                    #Extracts Info
                    this_info = line.split('"fooinfo">')[1].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #print(this_info)
                    
                    #Starts a loop to add move information to the dictitionary for the corresponding FORMS for individual form information
                    #Starts loop through the forms in this_form_list
                    for current_form in this_form_list:                    
                        #Finding the corresponding form within the dictionary
                        #Creating/resetting helpful loop counter:
                        form_counter = 1

                        #When dealing with the Standard form  (assumes standard form to be Form1)
                        if current_form == 'Standard' or current_form == 'Normal':
                            form_counter = 1
                            #print('yes')
                            #print(form_counter)

                        #When not dealing with standard form
                        if current_form != 'Standard' and current_form != 'Normal':
                            #Starting loop
                            while current_form not in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #When not a match, moves counter to next iteration
                                form_counter = form_counter + 1
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                            #When a match, does nothing (this is only used for debugging)
                            #if current_form in dex_pokemon['Forms']['Form'+str(form_counter)]['Name']:
                                #print(dex_pokemon['Forms']['Form'+str(form_counter)]['Name'])
                                #print('yes')
                                #print(form_counter)
                                #print(move_counter['Form'+str(form_counter)])
                                

                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms']['Form'+str(form_counter)]['Moves']['Move'+str(move_counter['Form'+str(form_counter)])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter['Form'+str(form_counter)] = move_counter['Form'+str(form_counter)] + 1
                        


            #When moves are not separated by FORMS, we assume ALL FORMS can leaarn the moves
            if individual_form_special_moves == 0:

                #Splits the line, first close to the Move Tutor title to avoid information from the following session (e.g. Max Moves, Special Moves for Rotom)
                #second, right in front of move name to separate line by moves
                split_line = line.split('">Attack Name</')[1].split('.shtml">')
                #Deletes first element from the list
                split_line.pop(0)

                #Creates a loop to use all elements of split_line, except for element 0
                for element in split_line:

                    #Extracts move information using line splitting

                    #1)For the Move Name:
                    #Extracts Name
                    this_name = element.split('</a></')[0]
                    #print(this_name)

                    #2)For the Move Type:
                    #Extracts Type
                    this_type = element.split('/type/')[1].split('.gif')[0].capitalize()
                    #print(this_type)

                    #3)For the Move Mode:
                    #Extracts Mode
                    this_mode = element.split('/type/')[2].split('.png')[0].capitalize()
                    #When it's a Status Move 
                    if this_mode == 'other' or this_mode == 'Other':
                        this_mode = 'Status'
                    #print(this_mode)

                    #4)For the Power:
                    #Extracts Power
                    this_power = element.split('"fooinfo">')[3].split('<')[0]
                    #print(this_power)

                    #5)For the Accuracy:
                    #Extracts Accuracy
                    this_accuracy = element.split('"fooinfo">')[4].split('<')[0]
                    #print(this_accuracy)

                    #6)For the PP:
                    #Extracts PP
                    this_pp = element.split('"fooinfo">')[5].split('<')[0]
                    #print(this_pp)

                    #7)For the Effect %:
                    #Extracts Effect %
                    this_effect = element.split('"fooinfo">')[6].split('<')[0]
                    #print(this_effect)

                    #8)For the Info:
                    #Extracts Info
                    this_info = element.split('"fooinfo">')[7].split('<')[0].replace('Pok&eacute;mon','Pokemon').replace('&#x2019;',"'")  #fixes bug with the text
                    #print(this_info)   

                    #Loops through all the forms, one by one
                    for current_form in dex_pokemon['Forms']:
                        #Adds move information to dictionary for current_form
                        #Sets up sub dictionary entry for a new move
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])] = {'Name':[], 'Number':['--'], 'Type':[], 'Mode':[], 'Power':[], 'Accuracy':[], 'Effect':[], 'PP':[], 'Method':[], 'Info':[]}                
                        #0)Adds Method to this set of moves:
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Method'].append(current_method)                
                        #1)Adds Move Name to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Name'].append(this_name)                
                        #2)Adds Move Type to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Type'].append(this_type)                
                        #3)Adds Move Mode to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Mode'].append(this_mode)                
                        #4)Adds Move Power to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Power'].append(this_power)                
                        #5)Adds Move Accuracy to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Accuracy'].append(this_accuracy)                
                        #6)Adds Move PP to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['PP'].append(this_pp)                        
                        #7)Adds Move Effect % to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Effect'].append(this_effect)                
                        #8)Adds Move Info to current form
                        dex_pokemon['Forms'][current_form]['Moves']['Move'+str(move_counter[current_form])]['Info'].append(this_info)                
                        
                        #Moves move_counter to next iteration for current form
                        move_counter[current_form] = move_counter[current_form] + 1
                        
    
    

    #####################

                    
#Final check to make sure ALL FORMS have assigned abilities
#This is very important for when MULTIPLE FORMS share the same abilities and the abilities are only stated once (e.g. Darmanitan and Rotom)
#Creates/resets helpful form counter
counter_forms_abilities = 1
#Starts to loop through dictionary
while counter_forms_abilities <= len(dex_pokemon['Forms']):
    #Checks if ability name entry is empty
    if dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'] == []:
        #If the entry is empty for current FORM, copies information from the previous FORM (assumes abilities to be the same between the two FORMS)
        dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Name'] = dex_pokemon['Forms']['Form'+str(counter_forms_abilities-1)]['Ability']['Name']
        dex_pokemon['Forms']['Form'+str(counter_forms_abilities)]['Ability']['Info'] = dex_pokemon['Forms']['Form'+str(counter_forms_abilities-1)]['Ability']['Info']

    #Moves to next iteration
    counter_forms_abilities = counter_forms_abilities + 1    



f.close()

print(dex_pokemon['Forms'])

In [ ]:

print(dex_pokemon['Forms']['Form'+str(2)]['Ability']['Name'])
print(dex_pokemon['Forms']['Form'+str(6)]['Name'])

print(form_name)

In [ ]:
print(dex_pokemon['Forms']['Form1']['Moves']['Move1']['Method'][0])

In [ ]:
#This script opens up the pokemon entry file extracted from Serebii and searches for the relevant information

dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/001_Bulbasaur.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/090_Shellder.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/142_Aerodactyl.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/179_Mareep.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/836_Boltund.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/151_Mew.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/894_Regieleki.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/025_Pikachu.pokedexswsh'


#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/038_Ninetales.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/052_Meowth.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/492_Shaymin.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/555_Darmanitan.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/479_Rotom.pokedexswsh'

#dex_entry = 'C:/Users/leogd/swsh_pokedex_Serebii/745_Lycanroc.pokedexswsh'


## ============================================================================================== ##

from pokemon_functions import*

dex_pokemon = load_dex_entry(dex_entry)


In [ ]:
print(dex_pokemon)

In [1]:
import os
from pokemon_functions import*

data_path = 'C:/Users/leogd/swsh_pokedex_Serebii/'

pokemon_available = []

dex_pokemon = {}


#Compiles list of files within the provided databse directory
files = next(os.walk(data_path))[2]
#print(files)

#Compiles a python list of available pokemon dex entries
for f in files:
    if f.endswith('.pokedexswsh'):  #pokemon files
        pokemon_available.append(f)#.replace('.pokedexswsh',''))
        
#Loads every pokemon dex enrty        
for pokemon in pokemon_available:
    #try:
    dex_pokemon[pokemon.replace('.pokedexswsh','')] = {}
    dex_pokemon[pokemon.replace('.pokedexswsh','')] = dict(load_dex_entry(data_path+pokemon))
    #except:
        #pass
    
    

Bulbasaur
001
Ivysaur
002
Venusaur
003
Charmander
004
Charmeleon
005
Charizard
006
Squirtle
007
Wartortle
008
Blastoise
009
Caterpie
010
Metapod
011
Butterfree
012
Weedle
013
Kakuna
014
Beedrill
015
Pidgey
016
Pidgeotto
017
Pidgeot
018
Rattata
019
Raticate
020
Spearow
021
Fearow
022
Ekans
023
Arbok
024
Pikachu
025
Raichu
026
Sandshrew
027
Sandslash
028
Nidoran F
029
Nidorina
030
Nidoqueen
031
Nidoran M
032
Nidorino
033
Nidoking
034
Clefairy
035
Clefable
036
Vulpix
037
Ninetales
038
Jigglypuff
039
Wigglytuff
040
Zubat
041
Golbat
042
Oddish
043
Gloom
044
Vileplume
045
Paras
046
Parasect
047
Venonat
048
Venomoth
049
Diglett
050
Dugtrio
051
Meowth
052
Persian
053
Psyduck
054
Golduck
055
Mankey
056
Primeape
057
Growlithe
058
Arcanine
059
Poliwag
060
Poliwhirl
061
Poliwrath
062
Abra
063
Kadabra
064
Alakazam
065
Machop
066
Machoke
067
Machamp
068
Bellsprout
069
Weepinbell
070
Victreebel
071
Tentacool
072
Tentacruel
073
Geodude
074
Graveler
075
Golem
076
Ponyta
077
Rapidash
078
Slowpoke
079
Sl

Noivern
715
Xerneas
716
Yveltal
717
Zygarde
718
Diancie
719
Volcanion
721
Rowlet
722
Dartrix
723
Decidueye
724
Litten
725
Torracat
726
Incineroar
727
Popplio
728
Brionne
729
Primarina
730
Grubbin
736
Charjabug
737
Vikavolt
738
Cutiefly
742
Ribombee
743
Rockruff
744
Lycanroc
745
Wishiwashi
746
Mareanie
747
Toxapex
748
Mudbray
749
Mudsdale
750
Dewpider
751
Araquanid
752
Fomantis
753
Lurantis
754
Morelull
755
Shiinotic
756
Salandit
757
Salazzle
758
Stufful
759
Bewear
760
Bounsweet
761
Steenee
762
Tsareena
763
Comfey
764
Oranguru
765
Passimian
766
Wimpod
767
Golisopod
768
Sandygast
769
Palossand
770
Pyukumuku
771
Type: Null
772
Silvally
773
Turtonator
776
Togedemaru
777
Mimikyu
778
Drampa
780
Dhelmise
781
Jangmo-o
782
Hakamo-o
783
Kommo-o
784
Tapu Koko
785
Tapu Lele
786
Tapu Bulu
787
Tapu Fini
788
Cosmog
789
Cosmoem
790
Solgaleo
791
Lunala
792
Nihilego
793
Buzzwole
794
Pheromosa
795
Xurkitree
796
Celesteela
797
Kartana
798
Guzzlord
799
Necrozma
800
Magearna
801
Marshadow
802
Poipole
803
Na

In [3]:
print(dex_pokemon['876_Indeedee'])

{'Species': ['Indeedee'], 'Dex': ['876'], 'Forms': {'Form1': {'Name': 'Male', 'Types': ['Psychic', 'Normal'], 'Ability': {'Name': ['Inner Focus', 'Synchronize', 'Psychic Surge (Hidden Ability)'], 'Info': ['This Pokemon will not flinch. Does not prevent flinching with Focus Punch. Also prevents stats from being lowered by the ability Intimidate', 'When this Pokemon becomes Poisoned, Paralyzed, or Burned, so does the opponent. However, Fire-type and Water Veil ability Pokemon cannot be Burned, Poison-type and Steel-type and Immunity ability Pokemon cannot be Poisoned, and Limber ability Pokemon cannot be Paralyzed.', 'Activates']}, 'Moves': {'Move1': {'Name': ['Stored Power'], 'Number': ['-'], 'Type': ['Psychic'], 'Mode': ['Special'], 'Power': ['20'], 'Accuracy': ['100'], 'Effect': ['--'], 'PP': ['10'], 'Method': ['Level up'], 'Info': ["The user attacks the target with stored power. The more the user's stats are raised, the greater the move's power."]}, 'Move2': {'Name': ['Play Nice'], '

In [ ]:
print(pokemon)

In [ ]:
#Checks broken dex enrties        
for pokemon in pokemon_available:
    if len(dex_pokemon[pokemon.replace('.pokedexswsh','')]) <= 1:
        print(pokemon.replace('.pokedexswsh',''))